In [1]:
import numpy as np
import pandas as pd
from scipy import optimize

import matplotlib.pyplot as plt
from aicsimageio import AICSImage
from tifffile import tifffile
import cv2
import multipletau
import czifile
from skimage import io

In [2]:
%pdb

Automatic pdb calling has been turned ON


In [3]:
# path = "/mnt/sda1/Sophie/2-KineticAnalysisData/2-Datas/02-Long-Movie/SnailShadow/"
# files = [
#     "Snail S 061222 E1_2",
#     "Snail S 061222 E2_1",
#     "Snail S 061222 E2_3", 
#     "Snail S 061222 E3_2", 
#     "Snail S 061222 E3_3", ]


path = "/mnt/sda1/Sophie/2-KineticAnalysisData/2-Datas/02-Long-Movie/SnailCoreTATA/"
files = [
#         "Snail CT 061222 E1_1",
#         "Snail CT 061222 E1_2",
#         "Snail CT 061222 E1_3",
#         "Snail CT 061222 E2_1",
        'Snail CT 071222 E2_1',
        'Snail CT 071222 E2_2',
        "Snail CT 071222 E3_1",
        "Snail CT 071222 E3_3",
        ]

track_ext = "_track.csv"
czi_ext = ".czi"

DELTA_T = 0.39
X_SIZE = 0.0920714
Z_SIZE = 0.5
def func_(x, T, c):
    return (( ((T-x)/(c*(T**2))) * np.heaviside((T-x),0.5)) )

In [4]:
for f in files:
    print(f)
    # load image
    img = czifile.imread(path+'/'+f+czi_ext)
    img = AICSImage(path+'/'+f+czi_ext)
    first_channel_data = img.get_image_data("TZYX", C=0, S=0,)
    
    
    #load csv
    track_file = pd.read_csv(path+'/'+f+track_ext)
    track_file.drop(index=[0,1,2], inplace=True)
    track_file['FRAME'] = pd.to_numeric(track_file["FRAME"])
    track_file['POSITION_X'] = pd.to_numeric(track_file["POSITION_X"])
    track_file['POSITION_Y'] = pd.to_numeric(track_file["POSITION_Y"])
    track_file['TRACK_ID'] = pd.to_numeric(track_file["TRACK_ID"])
    track_file['MEAN_INTENSITY_CH1'] = pd.to_numeric(track_file["MEAN_INTENSITY_CH1"])
    track_file['POSITION_T'] = pd.to_numeric(track_file["POSITION_T"])
    track_file['POSITION_Z'] = pd.to_numeric(track_file["POSITION_Z"])
    
    for id_ in np.unique(track_file['TRACK_ID']):

        x = track_file[track_file.TRACK_ID==id_].sort_values('FRAME')['POSITION_T'].values-min(track_file[track_file.TRACK_ID==id_].sort_values('FRAME')['POSITION_T'].values)
        y = track_file[track_file.TRACK_ID==id_].sort_values('FRAME')['MEAN_INTENSITY_CH1'].values-np.min(track_file[track_file.TRACK_ID==id_].sort_values('FRAME')['MEAN_INTENSITY_CH1'].values)
       
        
        
        mm = int(len(track_file[track_file.TRACK_ID==id_])/2-1)
        if (mm%2)==0: 
            autocor = multipletau.autocorrelate(y,
                                            m=mm,
                                            deltat=DELTA_T, 
                                            normalize=True)
        else: 
            autocor = multipletau.autocorrelate(y,
                                            m=mm+1,
                                            deltat=DELTA_T,
                                            normalize=True)



        popt, pcov = optimize.curve_fit(func_, 
                                    autocor.flatten()[0::2],
                                    autocor.flatten()[1::2], 
                                    method='lm' )
        if popt[0]>5:
            track = track_file[track_file.TRACK_ID==id_].sort_values('FRAME')
            for id_, val in track.iterrows():
                
                
                # Center coordinates
                center_coordinates = (int(val['POSITION_X']/X_SIZE), int(val['POSITION_Y']/X_SIZE))

                # Radius of circle
                radius = 2
                # Line thickness of 2 px
                thickness = 1
                image = cv2.circle(first_channel_data[val['FRAME']][int(val['POSITION_Z']/Z_SIZE)],
                                   center_coordinates,
                                   radius,
                                   np.max(track['MEAN_INTENSITY_CH1'])+5000,
                                   thickness)
    print("before tiff save")
    tifffile.imwrite(path+'/'+f+'_track.tiff',
                     first_channel_data.astype("uint16"),
                     imagej=True,
                     resolution=(1/X_SIZE, 1/X_SIZE),
                     metadata={'spacing': Z_SIZE,
                               'unit': 'um',
                               'axes': 'TZYX',
                               'hyperstack':True,
                               'channels':1,
                               'slices':first_channel_data.shape[1]})
    print('after tiff save')

Snail CT 071222 E2_1
before tiff save
after tiff save
Snail CT 071222 E2_2
before tiff save
after tiff save
Snail CT 071222 E3_1
before tiff save
after tiff save
Snail CT 071222 E3_3
before tiff save
after tiff save
